In [4]:

import io
import sys
import os
import string
import PIL
import cv2
import keras, tensorflow


from tkinter import *
import tkinter.font as tkFont
from PIL import ImageTk, Image
import pydicom
from tkinter.filedialog import askopenfilename
from tkinter import filedialog
import tkinter.messagebox as messagebox
from pathlib import Path

import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
from keras.models import Model, load_model

from skimage import color
from skimage import io
import numpy as np
from io import BytesIO
from PIL import Image,ImageOps
import base64
import tensorflow as tf



filePath = ""
pastfilePath = ""



In [5]:
    #variables
longitud, altura = 150, 150


modelo = 'model_CT.h5'
pesos_modelo = 'pesos_CT.h5'



cnn = load_model(modelo,compile=False)
cnn.load_weights(pesos_modelo)
cnn.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.Nadam(), metrics=['accuracy'])



In [ ]:
if __name__ == "__main__":

	root = Tk()
	root.resizable(0,0)
	root.title('Detector de Hemorragias Intracraneales')
	root.iconbitmap('C:\\Users\\ASUS\\Desktop\\Karla xd\\Number_Recognizer_ML-master\\Number_Recognizer_ML-master\\imagen\\Brain.ico')
    
    #Tamaño de la ventana
	w = 900
	h = 670
	x = 200
	y = 10

	respuesta =""
	root.geometry("%dx%d+%d+%d" % (w, h, x, y))
	

    
    #fuentes
	times1 = tkFont.Font(family='Times',size=14, weight='bold')
	times2 = tkFont.Font(family='Times',size=12, weight='bold',slant = 'italic')
	times3 = tkFont.Font(family='Times',size=16, weight='bold',slant = 'italic')
	times4 = tkFont.Font(family='Times',size=14, weight='bold',slant = 'italic')
	timestit = tkFont.Font(family='Times',size=18, weight='bold',slant = 'italic')


    #Titulo de la ventana
    
    
    #titulo de la simple page
	label = Label(root, text="DETECCIÓN DE HEMORRAGIAS INTRACRANEALES")
	label.pack(anchor=CENTER,fill=X)
	label.config(fg="white",    # Foreground
			bg='#439A97',   # Background
			font=timestit, height = 3) 

    
    #parte izquierda
	la = Label(root, text="Ruta de Imagen: ", font = times2,background = 'white')
	la.place(x=30, y=170)
    
    #nombreruta
	nombrear=StringVar(None)
	pathText =Entry(root, width=50,textvariable = nombrear, state= "readonly")
	pathText.place(x=20, y=230)
    #Button
	
    

    
	def convertir_a_png(archivo_dicom):
        
        
        #cargar el dicom
		ds= pydicom.dcmread(archivo_dicom)
    
        #extraer la matriz de pixeles
		matriz_pixeles = ds.pixel_array
        
		matriz_pixeles = matriz_pixeles.astype(np.uint8)
        
		imagen_bone = cv2.applyColorMap(matriz_pixeles, cv2.COLORMAP_BONE)
		imagen_pil = Image.fromarray(imagen_bone)
        
        #Archivo sin la extension
		nombre_archivo = os.path.splitext(archivo_dicom)[0]
		
        
        #guarda la imagen como archivo png
		archivo_png = nombre_archivo + '.png'
		
		imagen_pil.save(archivo_png)
        
        
		return archivo_png
   
	def buscar():
		global img_tk

    
		archivo = filedialog.askopenfilename(title='Buscar imagen',filetypes=(("archivos png", "*.png"),("archivos dicom", "*.dcm")))
		
		nombrear.set(archivo)
        
		extension = os.path.splitext(archivo)[1].lower()
		
		if extension == ".dcm":
			archivo_png = convertir_a_png(archivo)
			print(archivo_png)
			nombrear.set(archivo_png)
			img= Image.open(archivo_png)
            
		elif extension == ".png":
			img = Image.open(archivo)

        
        #guarda imagen
		img_tam = img.resize((300, 300))
		img_tk = ImageTk.PhotoImage(img_tam)
		
        
        
        #muestra de imagen
		labelimg.config(image=img_tk)
		return
    
    
    #boton de prediccion
	bu = Button(root, text='Cargar Imagen',borderwidth = 1,highlightbackground='black',command=buscar)
	bu.place(x=170, y=172)
    

	labelimg = Label(root, image="")
	labelimg.place(x=450, y=200)

			
	#------------------
	la2 = Label(root, text="Visualización de Imagen:", font = times2,background = 'white')
	la2.place(x=390, y=140)

	
	#------------------
    
    
	#------------------	------------------
    #prediccion
    
	def predict():
        
		cadena=nombrear.get()
		print(cadena)
        
		if cadena!="":
            
			#tomar la ruta
			ruta_path = Path(cadena)
			print(ruta_path)


            
			#lee la imagen y la normaliza
			img = cv2.imread(cadena, cv2.IMREAD_GRAYSCALE)
			img = cv2.resize(img, (longitud, altura))
			img = img_to_array(img)
			img = img / 255.0
			img = np.expand_dims(img, axis=0)
            
            

			#se predice la imagen en la cnn
			array= cnn.predict(img)
			etiqueta = ["Tiene ICH","NO TIENE ICH"]
            
            #
			clase_predicha = etiqueta[np.argmax(array)]
			probabilidad = np.max(array) * 100

			lapred.config(text=clase_predicha)
			laprob.config(text=f"PROBABILIDAD: {probabilidad:.2f}%")
            
		else:
			messagebox.showerror(message="Tiene que cargar una imagen", title="Error")
		return
    
    
	bu = Button(root, text='Detectar ICH',borderwidth = 5, highlightbackground='black', font = times1,command=predict)
	bu.place(x=85, y=290)

	lapred = Label(root, text="", font = times4, background = 'white')
	lapred.place(x=60, y=380)
            
            
	laprob = Label(root, text="", font = times4, background = 'white')
	laprob.place(x=60, y=420)
    

    
    
    
def on_closing():
    if messagebox.askokcancel("Salir", "¿Estas seguro que quieres salir?"):
        root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.configure(background='white')
root.mainloop()


C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_04463e63d.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_04463e63d.png
1/1 [==============================] - 1s 540ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_04463e63d.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_04463e63d.png
1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_4482eabc1.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_4482eabc1.png
1/1 [==============================] - 0s 47ms/step
C:/Users/

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\envs\proyecto\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_27124\3420002661.py", line 98, in buscar
    img_tam = img.resize((300, 300))
UnboundLocalError: local variable 'img' referenced before assignment



C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_4905f9035.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_4905f9035.png
1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_5048dfbb0.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_5048dfbb0.png
1/1 [==============================] - 0s 28ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_5178b5bd3.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_5178b5bd3.png
1/1 [==============================] - 0s 31ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\envs\proyecto\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_27124\3420002661.py", line 98, in buscar
    img_tam = img.resize((300, 300))
UnboundLocalError: local variable 'img' referenced before assignment


C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_5657b9743.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_5657b9743.png
1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_6293b3ac2.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_6293b3ac2.png
1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_4921a89d7.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_4921a89d7.png
1/1 [==============================] - 0s 31ms/step
C:/Users/A

1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/ich/ID_5058a13b1.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\ich\ID_5058a13b1.png
1/1 [==============================] - 0s 16ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/ich/ID_5410d5667.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\ich\ID_5410d5667.png
1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/ich/ID_5248cc1c5.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\ich\ID_5248cc1c5.png
1/1 [=================

1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/ich/ID_e465e1ee5.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\ich\ID_e465e1ee5.png
1/1 [==============================] - 0s 31ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_86985aad8.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_86985aad8.png
1/1 [==============================] - 0s 203ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/RNSA-2500-each-class/noich/ID_8698bcd86.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\RNSA-2500-each-class\noich\ID_8698bcd86.png
1/1 [========

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\envs\proyecto\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_27124\3420002661.py", line 98, in buscar
    img_tam = img.resize((300, 300))
UnboundLocalError: local variable 'img' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\envs\proyecto\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_27124\3420002661.py", line 98, in buscar
    img_tam = img.resize((300, 300))
UnboundLocalError: local variable 'img' referenced before assignment


C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/Imagenes de prueba/ICH/ID_5781c395f.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\Imagenes de prueba\ICH\ID_5781c395f.png
1/1 [==============================] - 0s 281ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/Imagenes de prueba/ICH/ID_5781c395f.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\Imagenes de prueba\ICH\ID_5781c395f.png
1/1 [==============================] - 0s 94ms/step
C:/Users/ASUS/Desktop/Karla xd/Number_Recognizer_ML-master/Number_Recognizer_ML-master/archive/Imagenes de prueba/NOICH/ID_04558b41c.png
C:\Users\ASUS\Desktop\Karla xd\Number_Recognizer_ML-master\Number_Recognizer_ML-master\archive\Imagenes de prueba\NOICH\ID_04558b41c.png
1/1 [==============================] - 0s 62ms/step
C:/Users/ASUS/Desktop/Karla x